# 流程增强

简单的 RAG 流程就像一条单行道，只能进行一次检索和生成。然而，现实世界的问题往往是复杂多变的，需要我们不断地探索和迭代。为了解决这个问题，我们需要对 RAG 流程进行增强，使其能够更灵活地适应不同的查询和场景。流程增强的目标是打破单行道的限制，建立一个动态的、可迭代的 RAG 流程，从而提高 RAG 系统的整体性能。



## 一、Self-RAG高级检索

Self-RAG代表了一种创新的框架，它利用自我反思的标记来对语言模型（LM）进行训练和调控。这个框架的核心流程可以划分为三个主要环节：信息检索、内容生成和内容评估。

![selfrag](./figures/selfrag.png)


1. 在信息检索环节，Self-RAG首先对检索标记进行解码，以判断当前情境下是否有必要进行信息的检索，并据此控制检索机制。如果确定需要检索，LM便会激活外部的检索模块，以便搜寻与当前任务相关的文档资料。

2. 进入内容生成阶段，若无需额外检索，LM将直接预测接下来的文本片段。若检索是必要的，LM会先生成一系列批评标记，这些标记的作用是评估已检索到的文档资料的相关性，并基于这些资料来构建后续的内容。

3. 在内容评估环节，LM进一步评价检索到的文档对生成内容的支持程度。最终，通过一个新的批评标记来对整体的响应效果进行评估。

![selftoken](./figures/selftoken.jpg)

在训练Self-RAG的过程中，涉及到三个关键的模型组件：检索器、批评家和生成器。首先，批评家模型会被训练，它利用检索器找到的文档和增强的反思标记来训练。接着，生成器LM通过标准的下一个标记预测任务进行训练，目的是掌握如何生成自然连贯的文本以及如何使用特殊的标记来检索或评估自身的生成内容。

在推理应用中，Self-RAG通过掌握生成反思标记的能力，能够为多种不同的下游任务或特定需求定制模型的行为，而无需对LMs进行额外的训练。它能够根据检索标记灵活地决定何时进行检索，让模型自行判断是否需要进行信息检索。此外，Self-RAG引入了多种细致的批评标记，用于评估生成内容的各种质量维度。在生成过程中，研究者采用了基于期望批评标记概率的线性插值方法，进行分段级别的束搜索（beam search），以便在每个时间节点确定最佳的续写方案

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "sk-你的sk"

首先构建一个字典，包含文件的八个标准问答对，key是问题，value是期望rag的答案

In [ ]:
qna_dict = {
        "介绍下SVM算法": "是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感知机",
        "用通俗的语言介绍下强化学习": "监督学习的特点是有一个“老师”来“监督”我们，告诉我们正确的结果是什么。在我们在小的时候，会有老师来教我们，本质上监督学习是一种知识的传递，但不能发现新的知识。对于人类整体而言，真正（甚至唯一）的知识来源是实践——也就是强化学习。比如神农尝百草，最早人类并不知道哪些草能治病，但是通\过尝试，就能学到新的知识。学习与决策者被称为智能体，与智能体交互的部分则称为环境。智能体与环境不断进行交互，具体而言，这一交互的过程可以看做是多个时刻，每一时刻，智能体根据环境的状态，依据一定的策略选择一个动作（这里的策略指的是从环境状态到智能体动作或者动作概率之间的映射），然后环境依据一定的状态转移概率转移到下一个状态，与此同时根据此时状态的好坏反馈给智能体一个奖励。智能体可以根据环境的反馈调整其策略，然后继续在环境中探索，最终学习到一个能够获得最多奖励的最优策略",
        "BN 和 LN 区别": "Batch Normalization 是对这批样本的同一维度特征做归一化， Layer Normalization 是对这单个样本的所有维度特征做归一化。区别：LN 中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN 中则针对不同神经元输入计算均值和方差，同一个 batch 中的输入拥有相同的均值和方差。所以，LN 不依赖于batch 的大小和输入 sequence 的长度，因此可以用于 batchsize 为 1 和 RNN 中 sequence 的 normalize 操作。",
        "讲讲self attention": "Self Attention 与传统的 Attention 机制非常的不同：传统的 Attention 是基于 source 端和 target 端的隐变量（hidden state）计算 Attention 的，得到的结果是源端的每个词与目标端每个词之间的依赖关系。但Self Attention 不同，它分别在 source 端和 target 端进行，仅与 source input 或者 target input 自身相关的 Self Attention，捕捉 source 端或 target 端自身的词与词之间的依赖关系；然后再把 source 端的得到的self Attention 加入到 target 端得到的 Attention 中，捕捉 source 端和 target 端词与词之间的依赖关系。因此，self Attention Attention 比传统的 Attention mechanism 效果要好，主要原因之一是，传统的Attention 机制忽略了源端或目标端句子中词与词之间的依赖关系，相对比，self Attention 可以不仅可以得到源端与目标端词与词之间的依赖关系，同时还可以有效获取源端或目标端自身词与词之间的依赖关系",
        "Bert 的预训练过程": "Bert 的预训练主要包含两个任务，MLM 和 NSP，Masked Language Model 任务可以理解为完形填空，随机 mask 每一个句子中 15%的词，用其上下文来做预测；Next Sentence Prediction 任务选择一些句子对 A 与 B，其中 50%的数据 B 是 A 的下一条句子，剩余 50%的数据 B 是语料库中随机选择的，学习其中的相关性。BERT 预训练阶段实际上是将上述两个任务结合起来，同时进行，然后将所有的 Loss 相加。",
        "GPT 与 Bert 的区别": "GPT 是单向模型，无法利用上下文信息，只能利用上文；而 BERT 是双向模型。GPT 是基于自回归模型，可以应用在 NLU 和 NLG 两大任务，而原生的 BERT 采用的基于自编码模型，只能完成 NLU 任务，无法直接应用在文本生成上面。",
        "pca 属于有监督还是无监督": "PCA 按有监督和无监督划分应该属于无监督学习，所以数据集有无 y 并不重要，只是改变样本 X 的属性(特征)维度。",
        "介绍 transformer 算法": "Transformer 本身是一个典型的 encoder-decoder 模型，Encoder 端和 Decoder 端均有 6 个 Block，Encoder 端的 Block 包括两个模块，多头 self-attention 模块以及一个前馈神经网络模块；Decoder 端的Block 包括三个模块，多头 self-attention 模块，多头 Encoder-Decoder attention 交互模块，以及一个前馈神经网络模块；需要注意：Encoder 端和 Decoder 端中的每个模块都有残差层和 Layer Normalization层。",
    }

构建评估prompt和评估函数：


意在让大模型判断生成的结果和期望的结果是否含义一致，在下面使用各种advanecd-rag方法时会经常调用， 主要使用Llama Packs，Llama Packs是对llamaindex的更加高级的封装，几行代码实现各种高级RAG，如果想看具体的流程也可以直接进入github查看源码

In [ ]:
from llama_index.core.prompts import PromptTemplate
import re
import nest_asyncio
nest_asyncio.apply()
eval_template = PromptTemplate("""
            输入:
            Question: {question}
            LLM-Answer: {llm_answer}
            Expected-Answer: {expected_answer}

            任务:
            比较LLM-Answer和Expected-Answer，确定它们是否传达类似的含义。

            输出:
            意思一致：如果LLM-Answer和Expected-Answer传达类似的含义。 
            意思不同：如果LLM-Answer和Expected-Answer传达完全不同的含义。

            示例： 
            Question：法国的首都是什么？ 
            LLM-Answer: 巴黎 
            Expected-Answer: 光之城

            输出:
            意思一致""")
    
def eval_llm_output(row, llm):
    ''' Eval llm_answer and expected_answer for the question with llm '''
    question = row['question']
    llm_answer = row['llm_answer']
    expected_answer = row['expected_answer']
    prompt_message = eval_template.format(question=question, llm_answer=llm_answer, expected_answer=expected_answer)
    answer = llm.complete(prompt_message).text
    return answer.strip()

初始化 llm

In [ ]:

from llama_index.llms.openai import OpenAI

llm = OpenAI()

载入需要查询的数据文件

In [ ]:
from llama_index.readers.file import PDFReader
from pathlib import Path

loader = PDFReader()
documents = loader.load_data(
        file=Path(f"./data/face.pdf")
    )


下载Self-RAG pack

In [26]:
from llama_index.core.llama_pack import download_llama_pack

# download and install dependencies
SelfRAGPack=download_llama_pack(
    "SelfRAGPack",
    "D:/AAAAAAAAAAAIGC/self_rag_pack",
)

In [27]:
from llama_index.core import Document, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
index = VectorStoreIndex.from_documents(documents)

# Setup a simple retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

下载模型

In [8]:
download_dir = "./tmpmodel"  # Replace
!pip3 install -q huggingface-hub
!huggingface-cli download m4r1/selfrag_llama2_7b-GGUF selfrag_llama2_7b.q4_k_m.gguf --local-dir {download_dir} --local-dir-use-symlinks False

./tmpmodel\selfrag_llama2_7b.q4_k_m.gguf


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/m4r1/selfrag_llama2_7b-GGUF/resolve/main/selfrag_llama2_7b.q4_k_m.gguf to C:\Users\Alienware\.cache\huggingface\hub\tmpx3m0nk9s


初始化 self-Rag 检索器

In [30]:
from pathlib import Path

model_path = "./tmpmodel/selfrag_llama2_7b.q4_k_m.gguf"
agent_pack = SelfRAGPack(
    model_path=model_path, retriever=retriever, verbose=True
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./tmpmodel/selfrag_llama2_7b.q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_cou

用 self-Rag 检索器进行检索。因为 self-Rag 会进行多次判断，所以整个检索的链路相对较长，但是效果会更好。

In [33]:
import pandas as pd

answers = {'question':[], 'llm_answer': [], 'expected_answer': []}

for question in qna_dict.keys():
    expected_answer = qna_dict[question]
    # result
    response = agent_pack.run(question)
    answers['question'].append(question)
    answers['llm_answer'].append(response.response)
    answers['expected_answer'].append(expected_answer)

answers_df = pd.DataFrame(answers)


llama_print_timings:        load time =    2904.69 ms
llama_print_timings:      sample time =      16.82 ms /    40 runs   (    0.42 ms per token,  2378.83 tokens per second)
llama_print_timings: prompt eval time =    2904.54 ms /    22 tokens (  132.02 ms per token,     7.57 tokens per second)
llama_print_timings:        eval time =    8948.68 ms /    39 runs   (  229.45 ms per token,     4.36 tokens per second)
llama_print_timings:       total time =   12115.75 ms /    61 tokens


Retrieval required
Received: 10 documents
Start evaluation


Llama.generate: prefix-match hit

llama_print_timings:        load time =    2904.69 ms
llama_print_timings:      sample time =      21.36 ms /    50 runs   (    0.43 ms per token,  2341.37 tokens per second)
llama_print_timings: prompt eval time =   79415.59 ms /   743 tokens (  106.89 ms per token,     9.36 tokens per second)
llama_print_timings:        eval time =   11974.64 ms /    49 runs   (  244.38 ms per token,     4.09 tokens per second)
llama_print_timings:       total time =   94319.48 ms /   792 tokens


Input: ### Instruction:
介绍下SVM算法

### Response:
[Retrieval]<paragraph>第4页共46页第十七篇：2022年4月10日百度机器学习方向暑期实习面试题6道
1、介绍下SVM算法
是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感
知机。
SVM可分为三种：
线性可分SVM
当训练数据线性可分时，通过最大化硬间隔（hardmargin）可以学习得到一个线性分类器，即硬间隔
SVM。
线性SVM
当训练数据不能线性可分但是近似线性可分时，通过最大化软间隔（softmargin）也可以学习到一个线
性分类器，即软间隔SVM。
非线性SVM
当训练数据线性不可分时，通过使用核技巧（kerneltrick）和最大化软间隔，可以学习到一个非线性
SVM。
SVM的的学习策略就是间隔最大化，可形式化为一个求解凸二次规划的问题，也等价于正则化的合页损
失函数的最小化问题。SVM的的学习算法就是求解凸二次规划的最优化算法。
SVM如何选择核函数
Linear核：主要用于线性可分的情形。参数少，速度快，对于一般数据，分类效果已经很理想了。
RBF核：主要用于线性不可分的情形。参数多，分类结果非常依赖于参数。有很多人是通过训练数据的交
叉验证来寻找合适的参数，不过这个过程比较耗时。
2、介绍下逻辑回归算法
逻辑回归是在数据服从伯努利分布的假设下，通过极大似然的方法，运用梯度下降法来求解参数，从而达
到将数据二分类的目的。
逻辑回归的优缺点</paragraph>
Prediction: [Relevant]逻辑回归是一种常见的二分类模型，它的基本模型是定义在特征空间上的一个最大的
Score: 0.9285435719763134
1/10 paragraphs done



Llama.generate: prefix-match hit

llama_print_timings:        load time =    2904.69 ms
llama_print_timings:      sample time =       4.18 ms /    19 runs   (    0.22 ms per token,  4547.63 tokens per second)
llama_print_timings: prompt eval time =   89390.15 ms /   861 tokens (  103.82 ms per token,     9.63 tokens per second)
llama_print_timings:        eval time =    3332.36 ms /    18 runs   (  185.13 ms per token,     5.40 tokens per second)
llama_print_timings:       total time =   95194.60 ms /   879 tokens


...

Input: ### Instruction:
介绍 transformer 算法

### Response:
[Retrieval]<paragraph>第39页共46页第二十八篇：京东科技NLP实习面试题10道
1、Bert里面为什么用layernormalization，而不用batchnormalization，分别讲一下这
两个啥意思。
BatchNormalization是对这批样本的同一维度特征做归一化，LayerNormalization是对这单个样本
的所有维度特征做归一化。
区别：LN中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN中则针
对不同神经元输入计算均值和方差，同一个batch中的输入拥有相同的均值和方差。所以，LN不依赖于
batch的大小和输入sequence的长度，因此可以用于batchsize为1和RNN中sequence的normalize操
作。
2.、Bert里面为什么Q，K，V要用三个不同的矩阵，用一个不是也行吗
如果使用相同的矩阵，相同量级的情况下，q和k进行点积的值会是最大的，进行softmax的加权平均
后，该词所占的比重会最大，使得其他词的比重很少，无法有效利用上下文信息来增强当前词的语义表示，而
使用不同的QKV后，会很大程度减轻上述的影响。
3、Bert和transformer讲一下
1bert只有transformer的encode结构，是生成语言模型
2bert加入了输入句子的mask机制，在输入的时候会随机mask
3模型接收两个句子作为输入，并且预测其中第二个句子是否在原始文档中也是后续句子可以做对话机
制的应答。
4在训练BERT模型时，MaskedLM和NextSentencePrediction是一起训练的，目标就是要最小化
两种策略的组合损失函数。
4、AUC指标讲一下
AUC：AUC是ROC曲线下面的面积，AUC可以解读为从所有正例中随机选取一个样本A，再从所有负
例中随机选取一个样本B，分类器将A判为正例的概率比将B判为正例的概率大的可能性。AUC反映的是分
类器对样本的排序能力。AUC越大，自然排序能力越好，即分类器将越多的正例排在负例之前。</paragraph>
Prediction: [Relevant]AUC是ROC曲线下面的面积，AUC可以解读为从

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2904.69 ms
llama_print_timings:      sample time =      10.06 ms /    23 runs   (    0.44 ms per token,  2286.06 tokens per second)
llama_print_timings: prompt eval time =  465203.92 ms /   831 tokens (  559.81 ms per token,     1.79 tokens per second)
llama_print_timings:        eval time =    7131.03 ms /    22 runs   (  324.14 ms per token,     3.09 tokens per second)
llama_print_timings:       total time =  476835.26 ms /   853 tokens


Input: ### Instruction:
介绍 transformer 算法

### Response:
[Retrieval]<paragraph>第16页共46页4、wide&deep模型为什么要有wide层结构，优缺点，如何改进？
wide&deep模型中的wide部分可以通过利用交叉特征引入非线性高效的实现记忆能力，但需要依赖人
工特征工程。
改进：DeepFM在Wide&Deep的基础上进行改进，不需要预训练FM得到隐向量，不需要人工特征工
程，能同时学习低阶和高阶的组合特征；FM模块和Deep模块共享FeatureEmbedding部分，可以更快的
训练，以及更精确的训练学习。
5、推荐领域GBDT+LR的做法了解吗？
GBDT+LR由两部分组成，其中GBDT用来对训练集提取特征作为新的训练输入数据，LR作为新训练输
入数据的分类器。GBDT+LR的提出意味着特征工程可以完全交由一个独立的模型来完成，模型的输入可以是
原始的特征向量，不必在特征工程上投入过多的人工筛选和模型设计的精力，真正实现了端到端的训练。
6、粗排有哪些指标？NDCG了解吗？
(1)NDCG，排序相似性的指标，看精排的排序结果和粗排有多相似
(2)粗排的召回率/重叠率，粗排的topk和精排的topk有多大占比。
计算DCG，计算公式如下：
其中，rel为这个排序list，结果i的一个等级得分；i是指结果i的当前位置序号；
​在搜索引擎中，rel等级得分，是由人工抽样数据，并且根据一定的规则打出来的等级得分。
步骤二：
计算IDCG（IdealDCG），即完美序的DCG；计算方式也同步骤1，只是排序序列不是由算法得出，而
是由人工对序列根据一定的评估准则排出来的最佳序列。
​步骤三：根据前面2个步骤的出来的结果，计算NDCG，计算公式如下：</paragraph>
Prediction: [Relevant]NDCG = DCG / (DCG + 0.5 * IIDCG)[No support / Contradictory][Utility:5]
Score: 1.4900611161061235
9/10 paragraphs done



Llama.generate: prefix-match hit

llama_print_timings:        load time =    2904.69 ms
llama_print_timings:      sample time =      23.32 ms /    50 runs   (    0.47 ms per token,  2144.45 tokens per second)
llama_print_timings: prompt eval time = 1916958.72 ms /   616 tokens ( 3111.95 ms per token,     0.32 tokens per second)
llama_print_timings:        eval time =   12368.01 ms /    49 runs   (  252.41 ms per token,     3.96 tokens per second)
llama_print_timings:       total time = 1933642.38 ms /   665 tokens


Input: ### Instruction:
介绍 transformer 算法

### Response:
[Retrieval]<paragraph>第35页共46页beamsearch是介于全局搜索和贪婪搜索之间。使用beamsize参数来限制在每一步保留下来的可能性
词的数量。beamsearch是在测试阶段为了获得更好准确性而采取的一种策略，在训练阶段无需使用。
相比暴力搜索的全局最优解，降低时间复杂度的方法就是寻找次优解，具体就是把搜索空间中的N减下
来，每一步计算完只保留K个(beamsize)最大的取值路径，这样时间复杂度降为O(K*N*T)，K取值一般比
N小很多。这样得到的虽然不是最优解，但是在seq2seq模型的推理预测中可以兼顾时间和效果。
5、你了解的文本表示方法有哪些
基于one-hot、tf-idf、textrank等的bag-of-words；
主题模型：LSA（SVD）、pLSA、LDA；
基于词向量的固定表征：word2vec、fastText、glove
基于词向量的动态表征：elmo、GPT、bert
对比如下：
One-hot表示：维度灾难、语义鸿沟；
矩阵分解（LSA）：利用全局语料特征，但SVD求解计算复杂度大；
基于NNLM/RNNLM的词向量：词向量为副产物，存在效率不高等问题；
word2vec、fastText：优化效率高，但是基于局部语料；
glove：基于全局预料，结合了LSA和word2vec的优点；
elmo、GPT、bert：动态特征。</paragraph>
Prediction: [Relevant]以上文本表示方法中，基于一-hot表示的方式，每个单词都被看作一个独立的特征，因此对应的特
Score: 0.5503470700438311
10/10 paragraphs done

End evaluation
Selected the best answer: [Relevant]Transformer 是一种深度学习模型，主要用于机器翻译和自然语言处理任务。[Fully supported][Utility:5]
Final answer: Transformer 是一种深度学习模型，主要用于机器翻译和自然语言处理任务。


查看 self-Rag 检索器的结果

In [38]:
with pd.option_context('display.max_colwidth', None):
    display(answers_df)
answers_df.to_excel('self_rag_pack_results.xlsx', index=False)

,question,llm_answer,expected_answer
0,介绍下SVM算法,这里介绍的是线性可分SVM。,是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感知机
1,用通俗的语言介绍下强化学习,好的,监督学习的特点是有一个“老师”来“监督”我们，告诉我们正确的结果是什么。在我们在小的时候，会有老师来教我们，本质上监督学习是一种知识的传递，但不能发现新的知识。对于人类整体而言，真正（甚至唯一）的知识来源是实践——也就是强化学习。比如神农尝百草，最早人类并不知道哪些草能治病，但是通\过尝试，就能学到新的知识。学习与决策者被称为智能体，与智能体交互的部分则称为环境。智能体与环境不断进行交互，具体而言，这一交互的过程可以看做是多个时刻，每一时刻，智能体根据环境的状态，依据一定的策略选择一个动作（这里的策略指的是从环境状态到智能体动作或者动作概率之间的映射），然后环境依据一定的状态转移概率转移到下一个状态，与此同时根据此时状态的好坏反馈给智能体一个奖励。智能体可以根据环境的反馈调整其策略，然后继续在环境中探索，最终学习到一个能够获得最多奖励的最优策略
2,BN 和 LN 区别,​,Batch Normalization 是对这批样本的同一维度特征做归一化， Layer Normalization 是对这单个样本的所有维度特征做归一化。区别：LN 中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN 中则针对不同神经元输入计算均值和方差，同一个 batch 中的输入拥有相同的均值和方差。所以，LN 不依赖于batch 的大小和输入 sequence 的长度，因此可以用于 batchsize 为 1 和 RNN 中 sequence 的 normalize 操作。
3,讲讲self attention,自我注意力（Self Attention）是一种神经语言模型中的一种重要概念，用于模型中的自我参与和内部连接。,Self Attention 与传统的 Attention 机制非常的不同：传统的 Attention 是基于 source 端和 target 端的隐变量（hidden state）计算 Attention 的，得到的结果是源端的每个词与目标端每个词之间的依赖关系。但Self Attention 不同，它分别在 source 端和 target 端进行，仅与 source input 或者 target input 自身相关的 Self Attention，捕捉 source 端或 target 端自身的词与词之间的依赖关系；然后再把 source 端的得到的self Attention 加入到 target 端得到的 Attention 中，捕捉 source 端和 target 端词与词之间的依赖关系。因此，self Attention Attention 比传统的 Attention mechanism 效果要好，主要原因之一是，传统的Attention 机制忽略了源端或目标端句子中词与词之间的依赖关系，相对比，self Attention 可以不仅可以得到源端与目标端词与词之间的依赖关系，同时还可以有效获取源端或目标端自身词与词之间的依赖关系
4,Bert 的预训练过程,因此，在实验中，我们需要确保我们的实验数据是一致的，以确保我们的结果是可信的。,Bert 的预训练主要包含两个任务，MLM 和 NSP，Masked Language Model 任务可以理解为完形填空，随机 mask 每一个句子中 15%的词，用其上下文来做预测；Next Sentence Prediction 任务选择一些句子对 A 与 B，其中 50%的数据 B 是 A 的下一条句子，剩余 50%的数据 B 是语料库中随机选择的，学习其中的相关性。BERT 预训练阶段实际上是将上述两个任务结合起来，同时进行，然后将所有的 Loss 相加。
5,GPT 与 Bert 的区别,Bert 是一种基于 Transformer 模型的语言模型，其中 GPT 是一种基于 Recurrent Neural Networks (RNN) 的语言模型。,GPT 是单向模型，无法利用上下文信息，只能利用上文；而 BERT 是双向模型。GPT 是基于自回归模型，可以应用在 NLU 和 NLG 两大任务，而原生的 BERT 采用的基于自编码模型，只能完成 NLU 任务，无法直接应用在文本生成上面。
6,pca 属于有监督还是无监督,W为输入大小，K为卷积核大小，P为padding大小，S为步幅。,PCA 按有监督和无监督划分应该属于无监督学习，所以数据集有无 y 并不重要，只是改变样本 X 的属性(特征)维度。
7,介绍 transformer 算法,Transformer 是一种深度学习模型，主要用于机器翻译和自然语言处理任务。,Transformer 本身是一个典型的 encoder-decoder 模型，Encoder 端和 Decoder 端均有 6 个 Block，Encoder 端的 Block 包括两个模块，多头 self-attention 模块以及一个前馈神经网络模块；Decoder 端的Block 包括三个模块，多头 self-attention 模块，多头 Encoder-Decoder attention 交互模块，以及一个前馈神经网络模块；需要注意：Encoder 端和 Decoder 端中的每个模块都有残差层和 Layer Normalization层。


## 二、查询路由和Multi-Document Agent相关概念

### 查询路由
查询路由是LLM驱动的关键决策环节，它决定了在接收到用户查询后应采取的最佳行动——可能的行动包括生成摘要、对特定数据索引执行搜索或尝试多种不同的路由策略，并将它们的输出汇总成一个统一的答案。

查询路由还负责选择最合适的数据存储位置来响应用户查询。这些数据存储位置可以是多种形式，如传统的向量存储、图形数据库、关系型数据库，或是分层次的索引系统。在处理包含多个文档的存储时，通常会利用两种不同的索引方式：摘要索引和文档块向量索引。

定义查询路由器涉及设定它可以采取的行动选项。

选择特定路由的过程是通过调用大型语言模型来实现的，其结果会按照预定义的格式返回，以便将查询定向到指定的索引。如果涉及到复杂的关联操作，这些查询还可能被发送到子链或其他智能体，如下文所述的多文档智能体方案所示。

LlamaIndex和LangChain都提供了对查询路由的支持。

### 智能体（Agent）
在智能体技术的实现上，尤其是在基于大型语言模型（LLM）的智能体构建中，LLM在智能体的智能化中扮演着至关重要的角色。这些智能体能够通过整合LLM与规划、记忆以及其他关键技术模块，执行复杂的任务。在此框架中，LLM充当核心处理单元或“大脑”，负责管理和执行为特定任务或响应用户查询所需的一系列操作。
智能体也是Langchain和LlamaIndex支持的一个概念，它从第一个LLM API发布之初就已存在。这个概念旨在为具备推理能力的LLM提供一套工具和任务。这些工具可能包括一些确定性的功能，如代码函数或外部API，甚至包括其他智能体——这种LLM之间的链接是LangChain名称的灵感来源。
智能体本身是一个复杂的技术，下面将继续基于智能体的多文档检索案例，并在下文将要介绍的RAG系统中发挥作用。
在LlamaIndex中，有一个OpenAIAgent类，它将这种高级逻辑与ChatEngine和QueryEngine类结合在一起，提供基于知识和上下文感知的聊天，以及在一个对话轮次中调用多个OpenAI函数的能力，这真正实现了智能代理行为。

### 多文档智能体方案
![mda](./figures/mda.png)

多文档智能体的方案是一个高度复杂的配置，它在每个文档上初始化一个Agent（OpenAIAgent），该智能体能进行文档摘要制作和传统问答机制的操作，还有一个顶层智能体，负责将查询分配到各个文档智能体，并综合形成最终的答案。

每个文档智能体都配备了两个工具：向量存储索引和摘要索引，它根据路由查询来决定使用哪一个。对于顶层智能体来说，所有文档智能体都是其工具。

该方案展示了一种高级RAG架构，其中每个智能体都进行多个决策的导航。这种方法的优势在于能够比较不同的解决方案或实体在不同文档及其摘要中的描述，以及传统的单个文档摘要和QA机制——这基本上涵盖了与文档集合聊天的最常见用例。

这种复杂配置的缺点在于，由于需要在智能体内部的大语言模型之间进行多次往返迭代，其运行速度较慢。值得一提的是，LLM调用通常是RAG管道中最耗时的操作，而搜索则是为了速度而优化的设计。因此，对于大型的多文档存储，建议考虑对这种方案进行简化，以便实现扩展。

下面将根据llamaindex官方示例进行讲解

In [ ]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-你的sk"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
)
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager

In [ ]:
wiki_titles = [
    "Toronto",
    "Seattle",
    "Chicago",
    "Boston",
    "Houston",
    "Tokyo",
    "Berlin",
    "Lisbon",
    "Paris",
    "London",
    "Atlanta",
    "Munich",
    "Shanghai",
    "Beijing",
    "Copenhagen",
    "Moscow",
    "Cairo",
    "Karachi",
]

将爬取到的文档存在本地 将在下面构建多文档智能体

In [ ]:
from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("./data/mutli_documents_data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w",encoding="utf-8") as fp:
        fp.write(wiki_text)

In [ ]:
# Load all wiki documents
city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"./data/mutli_documents_data/{wiki_title}.txt"]
    ).load_data()

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")


在下面小节中，将展示如何构建多文档智能体。首先为每个文档构建一个文档智能体，然后定义一个带有对象索引的顶层父级智能体。

#### 为每个文档构建文档智能体


1. 在本节中，为每个文档定义了“文档智能体”。

2. 为每个文档定义了一个向量索引（用于语义搜索）和摘要索引（用于生成摘要）。然后将这两个查询引擎转换为工具，并传递给调用OpenAI函数的智能体。

3. 该文档智能体可以动态选择在给定文档内执行语义搜索或生成摘要。

4. 我们为每个城市文档创建了一个单独的文档智能体。

In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.node_parser import SentenceSplitter
import os

node_parser = SentenceSplitter()

# Build agents dictionary
agents = {}
query_engines = {}

# this is for the baseline
all_nodes = []

for idx, wiki_title in enumerate(wiki_titles):
    nodes = node_parser.get_nodes_from_documents(city_docs[wiki_title])
    all_nodes.extend(nodes)

    if not os.path.exists(f"./data/mutli_documents_data/{wiki_title}"):
        # build vector index
        vector_index = VectorStoreIndex(nodes)
        vector_index.storage_context.persist(
            persist_dir=f"./data/mutli_documents_data/{wiki_title}"
        )
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/mutli_documents_data/{wiki_title}"),
        )

    # build summary index
    summary_index = SummaryIndex(nodes)
    # define query engines
    vector_query_engine = vector_index.as_query_engine(llm=llm)
    summary_query_engine = summary_index.as_query_engine(llm=llm)

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    "Useful for questions related to specific aspects of"
                    f" {wiki_title} (e.g. the history, arts and culture,"
                    " sports, demographics, or more)."
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    "Useful for any requests that require a holistic summary"
                    f" of EVERYTHING about {wiki_title}. For questions about"
                    " more specific sections, please use the vector_tool."
                ),
            ),
        ),
    ]

    # build agent
    function_llm = OpenAI(model="gpt-3.5-turbo")
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=f"""\
You are a specialized agent designed to answer queries about {wiki_title}.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
    )

    agents[wiki_title] = agent
    query_engines[wiki_title] = vector_index.as_query_engine(
        similarity_top_k=2
    )

#### 构建检索器支持的OpenAI智能体

下面构建了一个顶层智能体，它可以协调不同的文档智能体来回答用户的任何查询。

这个智能体将所有文档智能体作为工具。这个特定的智能体`RetrieverOpenAIAgent`在工具使用前执行工具检索（不同于默认智能体，后者尝试将所有工具放入提示中）。

在这里，使用了一个top-k检索器，也可以自定义其他工具检索方法！

In [ ]:
# define tool for each document agent
all_tools = []
for wiki_title in wiki_titles:
    wiki_summary = (
        f"This content contains Wikipedia articles about {wiki_title}. Use"
        f" this tool if you want to answer any questions about {wiki_title}.\n"
    )
    doc_tool = QueryEngineTool(
        query_engine=agents[wiki_title],
        metadata=ToolMetadata(
            name=f"tool_{wiki_title}",
            description=wiki_summary,
        ),
    )
    all_tools.append(doc_tool)

In [ ]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [ ]:
from llama_index.agent.openai import OpenAIAgent

top_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    system_prompt=""" \
You are an agent designed to answer queries about a set of given cities.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True,
)

#### 定义基线向量存储索引

作为比较的基准，我们定义了一个“朴素”的RAG流程，它将所有文档倾泻到一个单一的向量索引集合中。

我们设置 top_k = 4

In [ ]:
base_index = VectorStoreIndex(all_nodes)
base_query_engine = base_index.as_query_engine(similarity_top_k=4)

#### 执行示例查询

执行一些示例查询，范围从单个文档的问答/摘要到多个文档的问答/摘要。

In [ ]:
# should use Boston agent -> vector tool
response = top_agent.query("Tell me about the arts and culture in Boston")

Added user message to memory: Tell me about the arts and culture in Boston
=== Calling Function ===
Calling function: tool_Boston with args: {"input":"arts and culture"}
Added user message to memory: arts and culture
=== Calling Function ===
Calling function: vector_tool with args: {"input":"arts and culture in Boston"}
Got output: Boston has a rich cultural scene with a strong emphasis on arts and music. The city is home to several art museums and galleries, including the Museum of Fine Arts and the Isabella Stewart Gardner Museum. The Boston Symphony Orchestra, one of the "Big Five" American orchestras, is based in Symphony Hall. Boston also hosts various performing arts organizations, such as the Boston Ballet, Boston Lyric Opera Company, and the Handel and Haydn Society. Additionally, the city is known for its literary culture, with historical figures like Ralph Waldo Emerson and Nathaniel Hawthorne contributing to its reputation as "the intellectual capital of the United States."


In [ ]:
print(response)

Boston has a rich cultural scene with a strong emphasis on arts and music. The city is home to several art museums and galleries, including the Museum of Fine Arts and the Isabella Stewart Gardner Museum. The Boston Symphony Orchestra, one of the "Big Five" American orchestras, is based in Symphony Hall. Boston also hosts various performing arts organizations, such as the Boston Ballet, Boston Lyric Opera Company, and the Handel and Haydn Society. Additionally, the city is known for its literary culture, with historical figures like Ralph Waldo Emerson and Nathaniel Hawthorne contributing to its reputation as "the intellectual capital of the United States."


In [ ]:

# baseline
response = base_query_engine.query(
    "Tell me about the arts and culture in Boston"
)
print(str(response))

Boston has a rich arts and culture scene, with numerous art museums and galleries like the Museum of Fine Arts, Isabella Stewart Gardner Museum, and Institute of Contemporary Art. The city also boasts a vibrant music scene, highlighted by the Boston Symphony Orchestra, one of the "Big Five" American orchestras. Annual events such as the Boston Early Music Festival, Boston Arts Festival, and the Boston gay pride parade contribute to the city's cultural vibrancy. Boston also has a strong literary history, with renowned writers like Ralph Waldo Emerson, Nathaniel Hawthorne, and Henry Wadsworth Longfellow shaping the city's intellectual landscape.


In [ ]:

# should use Houston agent -> vector tool
response = top_agent.query(
    "Give me a summary of all the positive aspects of Houston"
)

In terms of history, the Premier League was established in 1992 and has since become one of the most popular and financially lucrative football leagues in the world. It is known for its competitive nature and has been dominated by clubs like Manchester United, Arsenal, Chelsea, Liverpool, and Manchester City. The league has also seen remarkable moments, such as Leicester City's title victory in the 2015-2016 season.

On the other hand, La Liga has a longer history, dating back to 1929. It has been home to some of the most successful and iconic clubs in football, including Real Madrid and Barcelona. The league has witnessed periods of dominance from various clubs and has seen intense rivalries between teams. Real Madrid has been particularly successful in the Champions League, winning numerous titles.

In terms of UEFA Champions League (UCL) performance, both leagues have had strong showings. English clubs have won a total of 15 UCL titles, making England the second-most successful coun

In [ ]:
# baseline
response = base_query_engine.query(
    "Give me a summary of all the positive aspects of Houston"
)
print(str(response))

Houston is a vibrant city with a robust economy fueled by exports, especially in petroleum products and oil-related industries. It has a diverse and expanding international community, showcasing various cultural events that highlight its multicultural essence. The city is famous for its culinary scene and restaurant culture, along with a lively arts and theater scene featuring numerous performing arts organizations and museums. Houston is a sports hub, hosting teams in major professional leagues, and offers a plethora of tourist attractions, parks, and recreational areas, making it an attractive destination for residents and visitors alike.


In [ ]:
# baseline: the response doesn't quite match the sources...
response.source_nodes[1].get_content()

'The city is home to the nation\'s third-largest concentration of consular offices, representing 92 countries.Many annual events celebrate the diverse cultures of Houston. The largest and longest-running is the annual Houston Livestock Show and Rodeo, held over 20 days from early to late March, and is the largest annual livestock show and rodeo in the world. Another large celebration is the annual night-time Houston Gay Pride Parade, held at the end of June. Other notable annual events include the Houston Greek Festival, Art Car Parade, the Houston Auto Show, the Houston International Festival, and the Bayou City Art Festival, which is considered to be one of the top five art festivals in the United States.Houston is highly regarded for its diverse food and restaurant culture. Houston received the official nickname of "Space City" in 1967 because it is the location of NASA\'s Lyndon B. Johnson Space Center. Other nicknames often used by locals include "Bayou City", "Clutch City", "Crus

In [ ]:

response = top_agent.query(
    "Tell the demographics of Houston, and then compare that with the"
    " demographics of Chicago"
)

Added user message to memory: Tell the demographics of Houston, and then compare that with the demographics of Chicago
=== Calling Function ===
Calling function: tool_Houston with args: {"input": "demographics"}
Added user message to memory: demographics
=== Calling Function ===
Calling function: summary_tool with args: {"input":"demographics"}
Got output: Houston is known for its diverse population, with various ethnic and religious backgrounds represented. The city has been described as the most racially and ethnically diverse major city in the U.S. Houston's population includes a large and growing international community, contributing to its status as a global city. Additionally, the city has a significant African American population, with historical ties to the post-Civil War era and the Great Migration.

Got output: Houston is known for its diverse population, with various ethnic and religious backgrounds represented. The city has been described as the most racially and ethnically

In [ ]:
# baseline
response = base_query_engine.query(
    "Tell me the differences between Shanghai and Beijing in terms of history"
    " and current economy"
)
print(str(response))

Shanghai's history is marked by international trade and foreign influence, especially during the 19th and early 20th centuries, shaping its identity as a major commercial and financial center. In contrast, Beijing has a more traditional background as the political and cultural heart of China, emphasizing governance and imperial legacy.

Currently, Shanghai is renowned for its emphasis on finance, innovation, and global trade, making a significant contribution to China's GDP. On the other hand, Beijing is known for its role as the political capital, housing government institutions and headquarters of state-owned enterprises, focusing on administrative functions and policy-making.


参考链接：https://docs.llamaindex.ai/en/v0.10.33/